In [ ]:
!pip install pyspark

In [1]:
# Import pyspark library
import pyspark

# Import spark session library
from pyspark.sql import SparkSession

In [3]:
# Create SparkSession object
spark = SparkSession\
        .builder\
        .appName("Decision_Tree_Classification")\
        .getOrCreate()

# Create DataFrame

In [4]:
# Read CSV data into a DataFrame using SparkSession
data1 = spark.read.format("csv")\
                .option("header", "true")\  # Use the first row as column headers
                .option("inferSchema", "true")\  # Infer the data types of columns
                .load("Social_Network_Ads.csv")  # Load data from the specified CSV file


In [5]:
data1.show()

+--------+------+---+---------------+---------+
| User ID|Gender|Age|EstimatedSalary|Purchased|
+--------+------+---+---------------+---------+
|15624510|  Male| 19|          19000|        0|
|15810944|  Male| 35|          20000|        0|
|15668575|Female| 26|          43000|        0|
|15603246|Female| 27|          57000|        0|
|15804002|  Male| 19|          76000|        0|
|15728773|  Male| 27|          58000|        0|
|15598044|Female| 27|          84000|        0|
|15694829|Female| 32|         150000|        1|
|15600575|  Male| 25|          33000|        0|
|15727311|Female| 35|          65000|        0|
|15570769|Female| 26|          80000|        0|
|15606274|Female| 26|          52000|        0|
|15746139|  Male| 20|          86000|        0|
|15704987|  Male| 32|          18000|        0|
|15628972|  Male| 18|          82000|        0|
|15697686|  Male| 29|          80000|        0|
|15733883|  Male| 47|          25000|        1|
|15617482|  Male| 45|          26000|   

# Verctorize the features

In [6]:
# Import necessary modules
from pyspark.ml.feature import *
from pyspark.ml.feature import VectorAssembler

In [8]:
# Create a VectorAssembler to combine specified input columns into a single feature column
vectorizer = VectorAssembler()  

# Set the input columns to be assembled into the features column
vectorizer.setInputCols(["Age", "EstimatedSalary"])  

# Set the name of the output feature column
vectorizer.setOutputCol("features")  

# Transform the DataFrame by assembling the specified input columns into the features column
data = vectorizer.transform(data1)  

# Display the first 10 rows of the transformed DataFrame
data.show(10, False)  


+--------+------+---+---------------+---------+---------------+
|User ID |Gender|Age|EstimatedSalary|Purchased|features       |
+--------+------+---+---------------+---------+---------------+
|15624510|Male  |19 |19000          |0        |[19.0,19000.0] |
|15810944|Male  |35 |20000          |0        |[35.0,20000.0] |
|15668575|Female|26 |43000          |0        |[26.0,43000.0] |
|15603246|Female|27 |57000          |0        |[27.0,57000.0] |
|15804002|Male  |19 |76000          |0        |[19.0,76000.0] |
|15728773|Male  |27 |58000          |0        |[27.0,58000.0] |
|15598044|Female|27 |84000          |0        |[27.0,84000.0] |
|15694829|Female|32 |150000         |1        |[32.0,150000.0]|
|15600575|Male  |25 |33000          |0        |[25.0,33000.0] |
|15727311|Female|35 |65000          |0        |[35.0,65000.0] |
+--------+------+---+---------------+---------+---------------+
only showing top 10 rows



# Data Scaling

In [9]:
# Import scaler library
from pyspark.ml.feature import MinMaxScaler

In [10]:
# Create a MinMaxScaler to scale the features column
scaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures")


In [13]:
# Fit the MinMaxScaler to the data and obtain a scaler model
scalerModel = scaler.fit(data)

# Transform the data using the fitted scaler model to obtain scaled features
scaledData = scalerModel.transform(data)

# Display the resulting DataFrame with scaled features
scaledData.show()


+--------+------+---+---------------+---------+---------------+--------------------+
| User ID|Gender|Age|EstimatedSalary|Purchased|       features|      scaledFeatures|
+--------+------+---+---------------+---------+---------------+--------------------+
|15624510|  Male| 19|          19000|        0| [19.0,19000.0]|[0.02380952380952...|
|15810944|  Male| 35|          20000|        0| [35.0,20000.0]|[0.40476190476190...|
|15668575|Female| 26|          43000|        0| [26.0,43000.0]|[0.19047619047619...|
|15603246|Female| 27|          57000|        0| [27.0,57000.0]|[0.21428571428571...|
|15804002|  Male| 19|          76000|        0| [19.0,76000.0]|[0.02380952380952...|
|15728773|  Male| 27|          58000|        0| [27.0,58000.0]|[0.21428571428571...|
|15598044|Female| 27|          84000|        0| [27.0,84000.0]|[0.21428571428571...|
|15694829|Female| 32|         150000|        1|[32.0,150000.0]|[0.33333333333333...|
|15600575|  Male| 25|          33000|        0| [25.0,33000.0]|[0

In [14]:

scaledData.select("scaledFeatures").take(2)


[Row(scaledFeatures=DenseVector([0.0238, 0.0296])),
 Row(scaledFeatures=DenseVector([0.4048, 0.037]))]

# Model Training

In [15]:
# Import necessary modules for creating a Spark ML Pipeline
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [16]:
# Index labels by adding metadata to the 'Purchased' column.
# Fit the StringIndexer on the entire dataset to include all labels in the index.
labelIndexer = StringIndexer(inputCol="Purchased", outputCol="indexedLabel").fit(scaledData)


In [17]:
# Automatically identify categorical features and index them.
# We specify maxCategories so that features with more than 4 distinct values are treated as continuous.
featureIndexer = VectorIndexer(inputCol="scaledFeatures", outputCol="indexedFeatures", maxCategories=4).fit(scaledData)


In [18]:
# Split the data into training and test sets with a 70-30 ratio (70% for training, 30% for testing)
(trainingData, testData) = scaledData.randomSplit([0.7, 0.3])


In [19]:
# Train a DecisionTree model with specified label and features columns
dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")


In [20]:
# Chain the label indexer, feature indexer, and decision tree in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dt])


In [21]:
# Train the model. This step also runs the label and feature indexers in the specified pipeline.
model = pipeline.fit(trainingData)


# Model Testing

In [22]:
# Make predictions using the trained model on the test data.
predictions = model.transform(testData)


In [23]:
# Select specific columns ('prediction', 'Purchased', 'scaledFeatures') from the predictions DataFrame and display the first 5 rows.
predictions.select("prediction", "Purchased", "scaledFeatures").show(5)


+----------+---------+--------------------+
|prediction|Purchased|      scaledFeatures|
+----------+---------+--------------------+
|       0.0|        0|[0.40476190476190...|
|       1.0|        1|[0.95238095238095...|
|       0.0|        0|[0.40476190476190...|
|       1.0|        1|[0.95238095238095...|
|       0.0|        0|[0.11904761904761...|
+----------+---------+--------------------+
only showing top 5 rows



# Model Summary

In [24]:
# Select (prediction, true label) and compute the test accuracy
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")

# Evaluate the accuracy of the predictions on the test data
accuracy = evaluator.evaluate(predictions)

# Print the test error, which is the complement of the accuracy
print("Test Error = %g" % (1.0 - accuracy))


Test Error = 0.0869565 


In [25]:
# Get the DecisionTree model from the trained pipeline model
treeModel = model.stages[2]

# Print a summary of the DecisionTree model
print(treeModel)


DecisionTreeClassificationModel: uid=DecisionTreeClassifier_eb30250fc606, depth=5, numNodes=31, numClasses=2, numFeatures=2


In [26]:
# Group predictions by 'Purchased' and 'prediction' columns, then count occurrences and display the result.
predictions.groupBy('Purchased', 'prediction').count().show()


+---------+----------+-----+
|Purchased|prediction|count|
+---------+----------+-----+
|        1|       0.0|    8|
|        0|       0.0|   66|
|        1|       1.0|   39|
|        0|       1.0|    2|
+---------+----------+-----+



In [27]:
# Calculate the elements of the confusion matrix
TN = predictions.filter('prediction = 0 AND Purchased = prediction').count()
TP = predictions.filter('prediction = 1 AND Purchased = prediction').count()
FN = predictions.filter('prediction = 0 AND Purchased = 1').count()
FP = predictions.filter('prediction = 1 AND Purchased = 0').count()

# Calculate model accuracy as the proportion of correct predictions
accuracy = (TN + TP) / (TN + TP + FN + FP)

# Print the model accuracy
print("Model Accuracy = %g" % (accuracy))


Model Accuracy = 0.913043 
